In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.model_selection import train_test_split

In [ ]:

# ---------------- STEP 1: Load and Preprocess Dataset ---------------- #

# Load dataset
file_path = "incremental_reasoning_dataset.jsonl"  # Update path if needed
df = pd.read_json(file_path, lines=True)

# Encode labels to numerical values
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Save label mappings for later use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # Maybe use BERT based model instead?
tokenizer = AutoTokenizer.from_pretrained(model_name)

def build_chat(p_user, p_assistant):
  messages = [
      {"role": "user", "content": p_user},
      {"role": "assistant", "content": p_assistant}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  return text

df["formatted_input"] = df.apply(lambda row: build_chat(row["input"], row["step_sequence"]), axis=1)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["formatted_input"], df["label"], test_size=0.1, random_state=42
)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=4096)

# Tokenize dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Load model with classification head
num_labels = len(label_encoder.classes_)  # Number of unique labels

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,  
    lora_alpha=16, 
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"])


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./mistake_classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=64,  
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    bf16=True, 
    gradient_checkpointing=True,  
    max_grad_norm=1.0,  
    optim="adamw_bf16",  
    save_total_limit=2,  
    report_to="none",  
    cuda_graphs=True,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()


model.save_pretrained("./mistake_classifier")
tokenizer.save_pretrained("./mistake_classifier")

print("Model fine-tuned and saved successfully!")